In [8]:
import os 
import random

import numpy as np
import pandas as pd 
import ujson
from tqdm import tqdm

In [9]:
run_path = "/home/jupyter/jointly_rec_and_search/datasets/rec_search/search/runs/bm25.train.run" 
qrels_path = "/home/jupyter/jointly_rec_and_search/datasets/rec_search/search/qrels.train.tsv" 
out_path = "/home/jupyter/jointly_rec_and_search/datasets/rec_search/search/1pos_1neg.train.tsv"

#neg_num = 1
#topk = 1000
qid_to_relpid = {}
rel_df = pd.read_csv(qrels_path, sep="\t", names=["qid", "q0", "pid", "score"])
for qid, group in tqdm(rel_df.groupby("qid")):
    relpid = group.sample(1).iloc[0]["pid"]
    qid_to_relpid[qid] = relpid

train_examples = []
df = pd.read_csv(run_path, sep=" ", names=["qid", "q0", "pid", "rank", "score", "model_name"])
for qid, group in tqdm(df.groupby("qid")):
    if len(group) <= 10:
        continue 
    relpid = qid_to_relpid[qid]
    negpid = group.sample(1).iloc[0]["pid"]
    train_examples.append((qid, relpid, negpid))

print("number of train_examples = {}".format(len(train_examples)))

with open(out_path, "w") as fout:
    for exp in train_examples:
        qid, relpid, negpid = exp 
        fout.write(f"{qid}\t{relpid}\t{negpid}\n")
    

100%|██████████| 211683/211683 [01:25<00:00, 2478.02it/s]


number of train_examples = 209735


In [10]:
! wc -l $qrels_path
! head -n 5 $qrels_path
print(50*"=")
! wc -l $out_path
! head -n 5 $out_path

870971 /home/jupyter/jointly_rec_and_search/datasets/rec_search/search/qrels.train.tsv
150882	Q0	8186	1
75048	Q0	478746	1
108656	Q0	551375	1
83347	Q0	1636968	1
106357	Q0	1911527	1
209735 /home/jupyter/jointly_rec_and_search/datasets/rec_search/search/1pos_1neg.train.tsv
2	2022667	937196
3	1569021	1625453
4	951670	711842
5	461041	1132563
6	269043	16784
